# Dependencies

In [63]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [64]:
import warnings 
warnings.filterwarnings('ignore')

# Agents

In [65]:
senior_support_staff = Agent(
    role='senior support representative',
    goal="Be the most friendly and helpful support representative in your team",
	backstory="""You work at Stripe (https://stripe.com) and are now working on providing support to {customer}, 
        a super important customer for your company You need to make sure that you provide the best support.
        Since you are a senior in this company, your answers should be holistic and cover all aspects of the 
        their doubts. Hence, make no assumptions.
    """,
    allow_delegation=False,
    verbose=True
)

In [66]:
quality_assurance_staff = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the best support quality assurance in your team",
	backstory="""You work at Stripe (https://stripe.com) and are now working with your team 
		on a request from {customer} ensuring that the support representative is 
		providing the best support possible. You need to make sure that the support representative 
        is providing full complete answers, and make no assumptions.
    """,
    allow_delegation=True,
	verbose=True
)

So here, we can see how the agent of quality assurance staff is **allowed to delegate** it's work to other agents. This way, it will choose the senior support staff to complete this task.

# Tools
We can now use some pre built tools from crewAI to work on this.

In [ ]:
from crewai_tools import ScrapeWebsiteTool

So now, let us create a document scrapper tool and give it access to the organisation of stripe.

In [68]:
document_tool = ScrapeWebsiteTool(
    base_url = "https://docs.stripe.com/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

## What makes a good tool?

1. Versitile
    1. Basically, it should be able to handle fuzzy inputs and give them out in structured output types, so that 
        they can interact well with traditional software systems.
2. Fault Tolerant
    1. If case of any exception in tools, they must not stop the crew. By default, crewAI implements this so that errors from the 
        tools are handled gracefully.
3. Caching
    1. It is best if there is a caching layer which works in a way so that more requests are not sent to the web, since most tools use 
        web connections to fetch output.

# Tasks

In [69]:
inquiry_resolution = Task(
    description="""
        {customer} just reached out with a super important ask:\n
	    {inquiry}\n\n
        {person} from {customer} is the one that reached out. 
		Make sure to use everything you know 
        to provide the best support possible.
		You must strive to provide a complete 
        and accurate response to the customer's inquiry.
    """,
    expected_output="""
	    A detailed, informative response to the 
        customer's inquiry that addresses 
        all aspects of their question.\n
        The response should include references 
        to everything you used to find the answer, 
        including external data or solutions. 
        Ensure the answer is complete, 
		leaving no questions unanswered, and maintain a helpful and friendly 
		tone throughout.""",
	tools=[document_tool],
    agent=senior_support_staff,
)

In [70]:
quality_assurance_review = Task(
    description="""
        Review the response drafted by the Senior Support Representative for {customer}'s inquiry. 
        Ensure that the answer is comprehensive, accurate, and adheres to the 
		high-quality standards expected for customer support.\n
        Verify that all parts of the customer's inquiry 
        have been addressed 
		thoroughly, with a helpful and friendly tone.\n
        Check for references and sources used to 
         find the information, 
		ensuring the response is well-supported and 
        leaves no questions unanswered.
    """,
    expected_output="""
        A final, detailed, and informative response 
        ready to be sent to the customer.\n
        This response should fully address the 
        customer's inquiry, incorporating all 
		relevant feedback and improvements.\n
		Don't be too formal, we are a chill and cool company 
	    but maintain a professional and friendly tone throughout.
    """,
    agent=quality_assurance_staff,
)


# Crew
Now we can go ahead and create the crew, which would consist of both these agents and tasks. Also, we must note that here, we would like to set memory as `True`.

In [71]:
crew = Crew(
    agents = [senior_support_staff,quality_assurance_staff],
    tasks = [inquiry_resolution,quality_assurance_review],
    memory = True,
    verbose = True 
)

2025-02-05 19:53:31,114 - 8270770752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Execution
Now, we can work on executing the crew to complete the task of writing a blog. 

In [72]:
inputs = {
    "customer": "100xDevs",
    "person": "Harkirat",
    "inquiry": "I need help with setting up a Stripe "
               "and kicking it off, specifically "
               "how can I handle disputes in stripe? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

# Agent: senior support representative
## Task: 
        100xDevs just reached out with a super important ask:

	    I need help with setting up a Stripe and kicking it off, specifically how can I handle disputes in stripe? Can you provide guidance?


        Harkirat from 100xDevs is the one that reached out. 
		Make sure to use everything you know 
        to provide the best support possible.
		You must strive to provide a complete 
        and accurate response to the customer's inquiry.
    


# Agent: senior support representative
## Thought: I need to gather the most accurate and up-to-date information on handling disputes in Stripe to ensure I provide comprehensive guidance to Harkirat from 100xDevs.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://stripe.com/docs/disputes\"}"
## Tool Output: 

Disputes and fraud | Stripe Documentation
Skip to content Overview Create account or Sign in The Stripe Docs logo Search the docs or ask a question / Crea

In [73]:
from IPython.display import Markdown,display 
display(Markdown(result.raw))

To set up a Stripe customer portal on your website and effectively handle disputes, please follow this detailed guide:

### Setting Up the Stripe Customer Portal

1. **Create a Stripe Account:**
   - If you haven't done so already, begin by creating a Stripe account at [Stripe's website](https://stripe.com). This will enable you to access all of Stripe's functionalities.

2. **Enable the Customer Portal:**
   - Log in to your Stripe Dashboard.
   - Navigate to the "Billing" section in the left-hand menu.
   - Under "Settings," find the "Customer portal" option.
   - Click on "Get started" and follow the prompts to enable the customer portal.

3. **Customize Portal Settings:**
   - Configure what your customers can do in the portal, such as viewing invoices, managing payment methods, reviewing subscription details, and checking their billing history.
   - You can also match the portal's appearance with your branding by customizing its look and feel.

4. **Implement the Portal into Your Website:**
   - To provide your customers with access to the portal, you'll need to create a unique session link for each customer.
   - Use Stripe's API to generate a session. Here’s a sample code snippet in Python:

   ```python
   import stripe

   stripe.api_key = 'your-secret-key'

   session = stripe.billing_portal.Session.create(
       customer='customer_id',
       return_url='https://your-website.com/account',
   )
   ```

5. **Redirect Customers to the Portal:**
   - Once you have created the session URL, you can redirect customers to the Stripe portal where they can manage their subscriptions and payment methods securely.

### Handling Disputes

1. **Understanding Disputes:**
   - A dispute arises when a customer questions a charge on their card and files a complaint with their bank. It’s important to respond to disputes promptly to mitigate potential revenue loss.

2. **Monitoring Disputes:**
   - You can monitor disputes in your Stripe Dashboard under the "Disputes" section. Stripe will send notifications via email whenever a new dispute is initiated.

3. **Responding to Disputes:**
   - To respond effectively to a dispute, you'll need to gather essential evidence. This could include:
     - Proof of delivery for physical goods.
     - Communication history with the customer, like emails or support chat transcripts.
     - Descriptions of the product or service related to the disputed charge.

4. **Submitting Evidence:**
   - You can upload supporting evidence directly through the Stripe Dashboard. Remember, disputes have response deadlines, so it’s crucial to act quickly.
   - Below is a sample code snippet for submitting evidence using Stripe's API:

   ```python
   dispute = stripe.Dispute.retrieve("dispute_id")
   dispute.evidence.add("receipt", evidence_file)
   dispute.save()
   ```

5. **Managing Customer Communication:**
   - Keep communication channels open with your customer throughout the dispute process. Informing them and addressing their concerns promptly can help maintain their trust.

6. **Learning from Disputes:**
   - After handling disputes, analyze patterns to identify common themes. This analysis can help you adjust policies and improve customer service, ultimately reducing the likelihood of future disputes.

### References
- For more detailed guidance and code examples, refer to the official Stripe documentation regarding the [Customer Portal](https://stripe.com/docs/billing/customer-portal) and [Dispute Handling](https://stripe.com/docs/disputes).

If you have specific questions or would like further assistance on any of these points, please feel free to reach out. I’m here to help and ensure you have everything you need to succeed!
```

In [77]:
inputs2 = {
    "customer": "100xDevs",
    "person": "Harkirat",
    "inquiry": "I need help with setting up a Stripe "
               "and kicking it off, specifically "
               "understanding Tax for software platforms, as my product is a SAAS"
               "Can you provide guidance?"
}
result2 = crew.kickoff(inputs=inputs2)

# Agent: senior support representative
## Task: 
        100xDevs just reached out with a super important ask:

	    I need help with setting up a Stripe and kicking it off, specifically understanding Tax for software platforms, as my product is a SAASCan you provide guidance?


        Harkirat from 100xDevs is the one that reached out. 
		Make sure to use everything you know 
        to provide the best support possible.
		You must strive to provide a complete 
        and accurate response to the customer's inquiry.
    


# Agent: senior support representative
## Thought: I need to gather detailed and accurate information on setting up Stripe for SaaS platforms and managing taxes for software products to support Harkirat from 100xDevs effectively.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://stripe.com/docs/tax\"}"
## Tool Output: 

Stripe Tax | Stripe Documentation
Skip to content Overview Create account or Sign in The Stripe Docs logo Search th

In [78]:
from IPython.display import Markdown,display 
display(Markdown(result2.raw))

To assist you in setting up Stripe for your SaaS platform and effectively managing taxes, here’s a detailed guide that encompasses both the setup of Stripe and compliance with tax regulations.

### Setting Up Stripe for Your SaaS Platform

1. **Create a Stripe Account:**
   - The first step is to create a Stripe account if you haven’t done so already by visiting [Stripe's website](https://stripe.com).

2. **Enable the Customer Portal:**
   - Log in to your Stripe Dashboard.
   - Navigate to the "Billing" section and find "Customer portal."
   - Enable the customer portal to allow your customers to manage their subscriptions easily.

3. **Customize Portal Settings:**
   - Define what your customers can do in the portal, such as viewing invoices or managing their payment methods.

4. **Implement Payments:**
   - Use the Stripe API to create payment flows. Stripe supports both one-time payments and subscriptions, perfect for SaaS products.

5. **Integrate Tax Management:**
   - **Automatic Tax Calculation:** Most importantly, since you are operating a software platform, you can enable Stripe Tax. This feature allows you to automatically calculate sales tax, VAT, and GST based on customer location and the nature of your product.
   - To set it up, configure your tax settings in the Stripe Dashboard. This enables Stripe to determine the appropriate tax rates for transactions automatically.
  
   - **Product Tax Codes:** Use product tax codes to clarify tax treatment and ensure compliance.

6. **Redirect Customers:**
   - After setup, create session links that redirect customers to the customer portal for subscription management.

### Handling Taxes for SaaS Products

1. **Understanding Tax Obligations:**
   - Since you're operating a SaaS platform, it's essential to understand various tax laws that might apply to your business based on where your customers are located. Common obligations include:
     - **United States:** Sales tax obligations vary by state.
     - **European Union:** VAT is applicable, and you may need to account for reverse charge mechanisms depending on customer type.
     - **Australia:** GST applies to sales made to Australian consumers.

2. **Setup in Stripe Tax:**
   - To get started with tax in Stripe, navigate to the **Tax** settings in your Dashboard. Here you can:
     - Register to collect taxes for different jurisdictions.
     - Manage tax registrations.
     - Use capability for calculating taxes automatically through various integration paths (e.g., Payment Links, Checkout Sessions).

3. **Tax Calculation:**
   - Stripe Tax calculates the appropriate rates based on multiple factors, including customer location and product type which is significant for SaaS offerings.

4. **File and Remit Taxes:**
   - Stripe aids in filing and remitting the tax collected to the respective tax authorities, simplifying compliance for you.

5. **Analyze Tax Information:**
   - Utilize reporting features to track collected tax amounts, aiding in your compliance and financial planning.

### Reference Materials
- For further guidance, please refer to the following sections of Stripe’s documentation:
  - **[Setting up Stripe Tax](https://stripe.com/docs/tax)**
  - **[Tax on Software Platforms](https://stripe.com/docs/tax#tax-software-products)**

If you have questions or need additional clarification on any specific aspect of this setup, feel free to reach out. I'm here to assist you every step of the way in ensuring a smooth setup and compliance process for your SaaS platform with Stripe!
```

# Conclusion
With this, we can see how we can create a team of agents which can work together to achieve a common goal. This acts an introduction to the concept of tools and how it can be used with agents to help agents give better outputs.